In [1]:
!pip install transformers

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv("output.csv")

In [3]:
train_df['task_1'].value_counts()

NOT    3591
HOF    2261
Name: task_1, dtype: int64

In [4]:
train_df['target']=train_df['task_1'].apply(lambda x:1 if x=='HOF' else 0)

In [5]:
train_df1=train_df[train_df['target']==1]
train_df1.shape

(2261, 6)

In [6]:
train_df2=train_df[train_df['target']==0]
train_df2.shape

(3591, 6)

In [7]:
train_df2=train_df2.sample(train_df1.shape[0])
train_df2.shape

(2261, 6)

In [8]:
train_df=pd.concat([train_df1,train_df2])
train_df.shape

(4522, 6)

In [9]:
train_df.head()

,text_id,text,task_1,task_2,task_3,target
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN,1
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN,1
11,hasoc_en_12,I don’t know how much more I can take! 45 is a...,HOF,HATE,TIN,1
15,hasoc_en_16,Good work @ICC keep going just destroy the who...,HOF,PRFN,TIN,1
23,hasoc_en_24,#ShameOnICC 1. ICC on Dhoni's gloves ...,HOF,HATE,TIN,1


In [10]:
# text processing function

import re
import string
def clean_text(text):
    # to lower case
    text = text.lower()
    # remove links
    text = re.sub('https:\/\/\S+', '', text)
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # remove next line
    text = re.sub(r'[^ \w\.]', '', text)
    # remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)

    return text
# Create a new column called "Text" for collecting clean text
train_df['Text'] = train_df.text.apply(lambda x: clean_text(x))

In [11]:
train_df.head()

,text_id,text,task_1,task_2,task_3,target,Text
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN,1,politico no we should remember very clearly th...
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN,1,ados trendingnow blacklivesmatter justice fuck...
11,hasoc_en_12,I don’t know how much more I can take! 45 is a...,HOF,HATE,TIN,1,i dont know how much more i can take is a com...
15,hasoc_en_16,Good work @ICC keep going just destroy the who...,HOF,PRFN,TIN,1,good work icc keep going just destroy the whol...
23,hasoc_en_24,#ShameOnICC 1. ICC on Dhoni's gloves ...,HOF,HATE,TIN,1,shameonicc icc on dhonis gloves ...


In [12]:
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [13]:
train_df.head()

,text_id,text,task_1,task_2,task_3,target,Text
4820,hasoc_en_4821,@KateBennett_DC @FLOTUS @CNN I hope you expose...,HOF,PRFN,TIN,1,katebennettdc flotus cnn i hope you expose her...
1075,hasoc_en_1076,Spectacular Win over Australia #TeamIndia Th...,NOT,NONE,NONE,0,spectacular win over australia teamindia the...
1938,hasoc_en_1939,@ICC Is truly islamic cricket council shame on...,HOF,HATE,TIN,1,icc is truly islamic cricket council shame on ...
3333,hasoc_en_3334,@narendramodi @ibusolih Sir what about #DhoniK...,NOT,NONE,NONE,0,narendramodi ibusolih sir what about dhonikeep...
272,hasoc_en_273,Candle march by doctors of BHU to support the ...,NOT,NONE,NONE,0,candle march by doctors of bhu to support the ...


In [14]:
train_df['target'].value_counts()

0    1839
1    1778
Name: target, dtype: int64

In [15]:
test_df['target'].value_counts()

1    483
0    422
Name: target, dtype: int64

In [16]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_df['Text'], train_df['target'], test_size=0.2, random_state=42)

In [17]:
test_texts, test_labels=(test_df['Text'], test_df['target'])

In [18]:
import torch
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

In [19]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def tokenize_texts(texts, labels):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels.to_numpy())
    dataset = TensorDataset(input_ids, attention_masks, labels)
    return dataset

train_dataset = tokenize_texts(train_texts, train_labels)
val_dataset = tokenize_texts(val_texts, val_labels)
test_dataset = tokenize_texts(test_texts, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [22]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6331491712707182
